## Accessing Sentinel-2 L2A data with the Planetary Computer STAC API

The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) is pre-configured to use your API key.


In [4]:
### Environment setup
import sys
sys.path.append('/content')
from src.fluvius import WaterData, WaterStation
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import geopandas as gpd
import fsspec
from pystac_client import Client
import planetary_computer as pc
import os

import matplotlib.pyplot as plt
# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

#################  set up ####################
data_source = 'itv'
container = f'{data_source}-data'
############## initial parameters ##############
if data_source == 'usgs':
    day_tolerance = 0 #reduce this for usgs-data
else:
    day_tolerance = 8 
cloud_thr = 80
buffer_distance = 500 # change this to increase chip size in meter 
################################################

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY'],
                 'connection_string': os.environ['CONNECTION_STRING']}

fs = fsspec.filesystem('az',\
                       account_name=storage_options['account_name'],\
                       account_key=storage_options['account_key'])   


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
ds = WaterData(data_source, container, storage_options)
ds.get_source_df()
ds.apply_buffer_to_points(buffer_distance)
ds.generate_map()
ds.plot_map

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [38]:
#full loop
for station in ds.df['site_no']:
    ds.get_station_data(station)
    ds.station[station].build_catalog()
    if ds.station[station].catalog is None:
        print('No matching images! Skipping...')
        continue
    else:
        ds.station[station].get_cloud_filtered_image_df(cloud_thr)
        ds.station[station].merge_image_df_with_samples(day_tolerance)
        ds.station[station].perform_chip_cloud_analysis()
        ds.station[station].get_reflectances()
        sstation = str(station).zfill(8)
        '''
        outfilename = f'az://{ds.container}/stations/{sstation}/{sstation}_processed.csv'
        ds.station[station].merged_df.to_csv(outfilename,index=False,storage_options=ds.storage_options)
        print(f'wrote csv {outfilename}')
        print('writing chips!')
        ds.station[station].write_tiles_to_blob(working_dirc='/tmp')
        '''

building catalog for station ITV1 with sentinel-2-l2a!
471 Items found
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/EE/2018/10/01/S2B_MSIL2A_20181001T135109_N0212_R024_T22MEE_20201009T070506.SAFE/GRANULE/L2A_T22MEE_A008201_20181001T135107/IMG_DATA/R20m/T22MEE_20181001T135109_SCL_20m.tif cloud chip error!
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/EE/2019/01/24/S2A_MSIL2A_20190124T135111_N0212_R024_T22MEE_20201008T041303.SAFE/GRANULE/L2A_T22MEE_A018754_20190124T135109/IMG_DATA/R20m/T22MEE_20190124T135111_SCL_20m.tif cloud chip error!
building catalog for station ITV2 with sentinel-2-l2a!
316 Items found
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/DE/2017/04/04/S2A_MSIL2A_20170404T135111_N0212_R024_T22MDE_20210209T044255.SAFE/GRANULE/L2A_T22MDE_A009316_20170404T135109/IMG_DATA/R20m/T22MDE_20170404T135111_SCL_20m.tif cloud chip error!
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/DE/2018/10/01/S2B_MSIL2A_20181001T13

https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/FE/2019/09/13/S2B_MSIL2A_20190913T134209_N0212_R124_T22MFE_20201004T124933.SAFE/GRANULE/L2A_T22MFE_A013163_20190913T134211/IMG_DATA/R20m/T22MFE_20190913T134209_SCL_20m.tif cloud chip error!
building catalog for station ITV10 with sentinel-2-l2a!
321 Items found
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/EE/2017/04/04/S2A_MSIL2A_20170404T135111_N0212_R024_T22MEE_20210209T044300.SAFE/GRANULE/L2A_T22MEE_A009316_20170404T135109/IMG_DATA/R20m/T22MEE_20170404T135111_SCL_20m.tif cloud chip error!
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/EE/2018/04/19/S2A_MSIL2A_20180419T135111_N0212_R024_T22MEE_20201013T000005.SAFE/GRANULE/L2A_T22MEE_A014750_20180419T135113/IMG_DATA/R20m/T22MEE_20180419T135111_SCL_20m.tif cloud chip error!
https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/22/M/EE/2018/10/01/S2B_MSIL2A_20181001T135109_N0212_R024_T22MEE_20201009T070506.SAFE/GRANULE/L2A_T22MEE_A008201